In [3]:
import numpy as np
import pandas as pd
import json
def id3(data, target, columns, tree,dataDomain,parent,dictionary,key=""):
    unique_targets = np.unique(data[target])

    if len(data)==0:
        dictionary[key]=(str(parent[target].value_counts().argmax())[:-2])
        return 
    elif len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if 1 in unique_targets:
            dictionary[key]=(str(1))
        else:
            dictionary[key]=(str(2))
        return 
    elif len(columns)==0:
        dictionary[key]=str(data[target].value_counts().argmax())[:-2]
        return 
    else:
        parent=data
        best_column = find_best_column(data, target, columns)
        # Insert code here that assigns the "column" and "median" fields to the node dictionary
        split_dict = []
        for i in dataDomain[best_column].keys():
            split_dict.append([i,data[data[best_column] == float(i)]])
        tree.insert(0,best_column)
        tree.insert(1,{})
        for name, split in split_dict:
            key=name
            tree[1][name]=[]
            dictionary=tree[1]
            id3(split, target, columns[columns!=best_column], tree[1][name],dataDomain,parent,dictionary,key)

def find_best_column(data,target,columns):
    best_columns=np.array([])
    for i in columns:
        gain = entropy(data[target])-mutualInfo(data[target],data[i])
        best_columns=np.append(best_columns, gain)
    maxIndex=np.argmax(best_columns)
    bestColumn=columns[maxIndex]
    return bestColumn



def mutualInfo(r1, r2):
    mutualDep=0
    S=len(r1)
    entropyR1=entropy(r1)
    for i in np.unique(r1):
        for j in np.unique(r2):
            Sab=len(r1[r2==j][r1[r2==j]==i])
            Sb=len(r2[r2==j])
            mutualDep+=(-(Sab)/S)*np.log2((Sab)/(Sb))
    return mutualDep


#Calculate the entropy for a, which is a set of values for a single feature.
def entropy(a):
    lenOfUnqElem=[]
    unique_elem=np.unique(a)
    for i in unique_elem:
        lenUniq=len(a[a==i])
        lenOfUnqElem.append(lenUniq)
    entropyS=0
    len_a=len(a)
    for j in lenOfUnqElem:
        entropyS+=((-j/len_a)*np.log2(j/len_a))
    return entropyS

    #Fill in the missing part here


In [4]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = []
data=np.loadtxt("train.txt")
data=data.T
data=pd.DataFrame(data,columns=["RISK","AGE", "CRED_HIS","INCOME","RACE","HEALTH"])
with open("deDomain.txt","r") as g:
    dataDomain=json.load(g)
id3(data, "RISK", np.array(["AGE", "CRED_HIS","RACE","INCOME","HEALTH"]),tree,dataDomain,parent=None,dictionary=None)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log2
/usr/lib/python3/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel_launcher.py:18: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:8: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


In [5]:
tree

['CRED_HIS',
 {'1': ['AGE',
   {'1': ['INCOME',
     {'1': ['HEALTH',
       {'1': ['RACE', {'1': '2', '2': '2', '3': '2'}],
        '2': ['RACE', {'1': '2', '2': '2', '3': '2'}]}],
      '2': '2'}],
    '2': ['HEALTH',
     {'1': ['RACE',
       {'1': '2', '2': ['INCOME', {'1': '2', '2': '2'}], '3': '2'}],
      '2': ['RACE',
       {'1': ['INCOME', {'1': '2', '2': '2'}],
        '2': '2',
        '3': ['INCOME', {'1': '2', '2': '2'}]}]}],
    '3': ['RACE',
     {'1': ['HEALTH', {'1': ['INCOME', {'1': '2', '2': '2'}], '2': '2'}],
      '2': ['HEALTH', {'1': ['INCOME', {'1': '1', '2': '1'}], '2': '1'}],
      '3': ['HEALTH', {'1': '2', '2': '1'}]}]}],
  '2': ['RACE',
   {'1': ['AGE',
     {'1': ['INCOME', {'1': '1', '2': ['HEALTH', {'1': '1', '2': '1'}]}],
      '2': ['INCOME', {'1': '1', '2': ['HEALTH', {'1': '2', '2': '1'}]}],
      '3': '1'}],
    '2': '1',
    '3': ['AGE',
     {'1': '1',
      '2': '1',
      '3': ['INCOME', {'1': '1', '2': ['HEALTH', {'1': '1', '2': '1'}]}]}]}]}]

In [6]:
with open("treeFileFull.txt","w") as f:
    json.dump(tree,f)

In [7]:
import numpy as np
import json


#Draw the (sub)decision tree rooted at tr. ls recovers the english
#names for the values in each domain, pre gives the number of blank
#spaces preceding each level, and f is the file where the drawing
#will be stored
def dispTree(tr,ls,sp,sb,f):
    if type(tr) != list:
        f.write(ls['RISK'][str(tr)])
        return
    f.write(tr[0]+'\n') #attribute selected in current node
    j = 0
    for v in tr[1]:#values in the domain of this attribute
        sp1 = sp.copy()
        sb1 = sb.copy()
        j += 1
        for k in range(len(sp1)):
            f.write(' '+sb1[k])
            for p in range(sp1[k]):
                f.write(' ')
        f.write(' !--'+ls[tr[0]][v]+'--')
        if list == type(tr[1][v]):
            sk = len(ls[tr[0]][v]) + 5
            sp1.append(sk)
            if j < len(tr[1]):
                sb1.append('!')
            else:
                sb1.append(' ')
            dispTree(tr[1][v],ls,sp1,sb1,f)
        else:
            f.write(ls['RISK'][str(tr[1][v])]+'\n')

def showIt(fname):
    msg = input('Plot un-pruned tree. Plot (Y/N)?')
    if msg == 'Y':
        with open('deDomain.txt') as f:
            m = json.load(f)
        with open(fname) as f:
            tr = json.load(f)
        with open('treePicFull.txt', 'w') as f:
            f.write('')
        with open('treePicFull.txt','a+') as f:
            dispTree(tr,m,[],[],f)
showIt("treeFileFull.txt")

Plot un-pruned tree. Plot (Y/N)?


In [8]:
data[(data["CRED_HIS"]==1)&(data["AGE"]==1)&(data["INCOME"]==2)]

,RISK,AGE,CRED_HIS,INCOME,RACE,HEALTH
115,2.0,1.0,1.0,2.0,1.0,1.0
130,2.0,1.0,1.0,2.0,3.0,1.0


In [9]:
l=

SyntaxError: invalid syntax (<ipython-input-9-a3e8ac245b24>, line 1)

In [ ]:
print(data[(data["CRED_HIS"]==1)&(data["AGE"]==3)&(data["HEALTH"]==1)&(data["RACE"]==1)])

In [ ]:
data["RISK"].value_counts()

In [ ]:
np.random.randint(1,3)

In [ ]:
l=[0]

In [ ]:
l.insert(1,{})

In [ ]:
l

In [ ]:
l[1]

In [ ]:
l[1]["gg"]

In [ ]:
d={1:[]}

In [ ]:
d[1]=5

In [ ]:
d

In [ ]:
import numpy as np
import pandas as pd
import json
def id3(data, target, columns, tree,dataDomain,parent):
    unique_targets = np.unique(data[target])

    if len(data)==0:
        tree[0]=str(parent[target].value_counts().argmax())[:-2]
        return 
    elif len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if 1 in unique_targets:
            tree[0]=str(1)
        else:
            tree[0]=str(2)
        return
    elif len(columns)==0:
        tree[0]=str(data[target].value_counts().argmax())[:-2]
        return
    else:
        parent=data
        best_column = find_best_column(data, target, columns)
        # Insert code here that assigns the "column" and "median" fields to the node dictionary
        split_dict = []
        for i in dataDomain[best_column].keys():
            split_dict.append([i,data[data[best_column] == float(i)]])
        tree[0]=best_column
        print(best_column)
        for name, split in split_dict:
            tree[1][name] = [None,{}]
            id3(split, target, columns[columns!=best_column], tree[1][name],dataDomain,parent)

def find_best_column(data,target,columns):
    best_columns=np.array([])
    for i in columns:
        gain = entropy(data[target])-mutualInfo(data[target],data[i])
        best_columns=np.append(best_columns, gain)
    maxIndex=np.argmax(best_columns)
    bestColumn=columns[maxIndex]
    return bestColumn



def mutualInfo(r1, r2):
    mutualDep=0
    S=len(r1)
    entropyR1=entropy(r1)
    for i in np.unique(r1):
        for j in np.unique(r2):
            Sab=len(r1[r2==j][r1[r2==j]==i])
            Sb=len(r2[r2==j])
            mutualDep+=(-(Sab)/S)*np.log2((Sab)/(Sb))
    return mutualDep


#Calculate the entropy for a, which is a set of values for a single feature.
def entropy(a):
    lenOfUnqElem=[]
    unique_elem=np.unique(a)
    for i in unique_elem:
        lenUniq=len(a[a==i])
        lenOfUnqElem.append(lenUniq)
    entropyS=0
    len_a=len(a)
    for j in lenOfUnqElem:
        entropyS+=((-j/len_a)*np.log2(j/len_a))
    return entropyS

    #Fill in the missing part here
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later

tree = [None,{}]
data=np.loadtxt("train.txt")
data=data.T
data=pd.DataFrame(data,columns=["RISK","AGE", "CRED_HIS","INCOME","RACE","HEALTH"])
with open("deDomain.txt","r") as g:
    dataDomain=json.load(g)
id3(data, "RISK", np.array(["AGE", "CRED_HIS","INCOME","RACE","HEALTH"]),tree,dataDomain,parent=None)
with open("treeFileFull.txt","w") as f:
    json.dump(tree,f)

#names for the values in each domain, pre gives the number of blank
#spaces preceding each level, and f is the file where the drawing
#will be stored
def dispTree(tr,ls,sp,sb,f):
    if type(tr) != list:
        f.write(ls['RISK'][str(tr)])
        return
    f.write(tr[0]+'\n') #attribute selected in current node
    j = 0
    for v in tr[1]:#values in the domain of this attribute
        sp1 = sp.copy()
        sb1 = sb.copy()
        j += 1
        for k in range(len(sp1)):
            f.write(' '+sb1[k])
            for p in range(sp1[k]):
                f.write(' ')
        f.write(' !--'+ls[tr[0]][v]+'--')
        if list == type(tr[1][v]):
            sk = len(ls[tr[0]][v]) + 5
            sp1.append(sk)
            if j < len(tr[1]):
                sb1.append('!')
            else:
                sb1.append(' ')
            dispTree(tr[1][v],ls,sp1,sb1,f)
        else:
            f.write(ls['RISK'][str(tr[1][v])]+'\n')

def showIt(fname):
    msg = input('Plot un-pruned tree. Plot (Y/N)?')
    if msg == 'Y':
        with open('deDomain.txt') as f:
            m = json.load(f)
        with open(fname) as f:
            tr = json.load(f)
        with open('treePicFull.txt', 'w') as f:
            f.write('')
        with open('treePicFull.txt','a+') as f:
            dispTree(tr,m,[],[],f)
showIt("treeFileFull.txt")
tree

In [ ]:
data.head()

In [10]:
y=data["RISK"].astype(str)

In [11]:
x=data[data.columns[data.columns!="RISK"]]

In [12]:
for i in x.columns:
    x[i]=x[i].astype(str)
x.dtypes

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


AGE         object
CRED_HIS    object
INCOME      object
RACE        object
HEALTH      object
dtype: object

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
dt=DecisionTreeClassifier(criterion='entropy')
dt.fit(x.values,y.values)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
dt.get_depth()

7

In [16]:
import pydot
from IPython import display

from sklearn import tree
dot_data = StringIO()
tree.export_graphviz(dt, out_file=dot_data)
dot_data.getvalue()
(graph, ) = pydot.graph_from_dot_data(dot_data.getvalue())
print('"graph" is object of type {}'.format(type(graph)))

display.Image(graph.create_png())

NameError: name 'StringIO' is not defined

In [26]:
def mutualInfo1(r1, r2):
    mutualDep=0
    S=len(r1)
    entropyR1=entropy(r1)
    for i in np.unique(r1):
        for j in np.unique(r2):
            Sab=len(r1[r2==j][r1[r2==j]==i])
            print("SA",Sab)
            Sb=len(r2[r2==j])
            print("Sb",Sb)
            if(Sab!=0):
                mutualDep+=(-(Sab)/S)*np.log2((Sab)/(Sb))
    return mutualDep


#Calculate the entropy for a, which is a set of values for a single feature.
def entropy1(a):
    lenOfUnqElem=[]
    unique_elem=np.unique(a)
    for i in unique_elem:
        lenUniq=len(a[a==i])
        lenOfUnqElem.append(lenUniq)
    entropyS=0
    len_a=len(a)
    for j in lenOfUnqElem:
        entropyS+=((-j/len_a)*np.log2(j/len_a))
    return entropyS

In [27]:
new=data[data["CRED_HIS"]==2]
new.tail()

,RISK,AGE,CRED_HIS,INCOME,RACE,HEALTH
111,2.0,3.0,2.0,2.0,3.0,1.0
116,2.0,1.0,2.0,2.0,1.0,2.0
118,2.0,2.0,2.0,2.0,1.0,2.0
120,2.0,2.0,2.0,2.0,1.0,1.0
124,2.0,2.0,2.0,2.0,1.0,2.0


In [28]:
def find_best_column1(data,target,columns):
    best_columns=np.array([])
    for i in columns:
        gain = entropy1(data[target])-mutualInfo1(data[target],data[i])
        best_columns=np.append(best_columns, gain)
    maxIndex=np.argmax(best_columns)
    bestColumn=columns[maxIndex]
    return best_columns
find_best_column1(new,"RISK",["AGE","INCOME","RACE","HEALTH"])

SA 16
Sb 17
SA 14
Sb 17
SA 48
Sb 49
SA 1
Sb 17
SA 3
Sb 17
SA 1
Sb 49
SA 15
Sb 15
SA 63
Sb 68
SA 0
Sb 15
SA 5
Sb 68
SA 30
Sb 34
SA 29
Sb 29
SA 19
Sb 20
SA 4
Sb 34
SA 0
Sb 29
SA 1
Sb 20
SA 33
Sb 35
SA 45
Sb 48
SA 2
Sb 35
SA 3
Sb 48


array([3.97445390e-02, 1.79284543e-02, 4.53289323e-02, 8.95952318e-05])